In [21]:
import json
import re

from utils import query

from sumeval.metrics.rouge import RougeCalculator
import numpy as np

## Build Lookup Files

In [22]:
db_file = '../src/data/mimic_iii_processed/mimic.db'
model = query(db_file)
(db_meta, db_tabs, db_head) = model._load_db(db_file)


for tb in db_meta:
    for hd in db_meta[tb]:
        mysql = 'select distinct {} from {}'.format(hd, tb)
        myres = model.execute_sql(mysql).fetchall()
        myres = list({k[0]: {} for k in myres if not k[0] == None})
        db_meta[tb][hd] = myres

In [23]:
headerDic = []
for tb in db_meta:
    for hd in db_meta[tb]:
        headerDic.append('.'.join([tb,hd]).lower())

In [24]:
generated_sql_file = open('../src/data/generated-sql/predicted-template-sql.json', 'r')

total_samples = 0
for _ in generated_sql_file:
    total_samples += 1

generated_sql_file.close()

print(f"Total samples: {total_samples}")

Total samples: 1000


## Logic Form Accuracy:

In [25]:
cnt = 0
sql_rec_logic = []
generated_sql_file = open('../src/data/generated-sql/predicted-template-sql.json', 'r')

for line in generated_sql_file:
    line = json.loads(line)
    pred = re.split('<stop>', line['pred_sql'])[0]
    real_sql = line['input_sql']

    predArr = re.split("where", pred)
    predAgg = re.split("\s", predArr[0])
    predAgg = list(filter(None, predAgg))
    predAgg2 = []
    for k in range(len(predAgg)-1):
        if predAgg[k] in headerDic and predAgg[k+1] in headerDic:
            predAgg2.append(predAgg[k] + ',')
        else:
            predAgg2.append(predAgg[k])
    predAgg2.append(predAgg[-1])
    predAgg = ' '.join(predAgg2)
    
    predCon = re.split("and", predArr[1])
    predConNew = []
    k = 0
    while k < len(predCon):
        if "=" in predCon[k] or "<" in predCon[k] or ">" in predCon[k] or ">" in predCon[k]:
            predConNew.append(predCon[k])
        else:
            if(len(predConNew) > 0):
                predConNew[-1] += " and " + predCon[k]
            k += 1
        k += 1
    for k in range(len(predConNew)):
        if "=" in predConNew[k]:
            conOp = "="
        if ">" in predConNew[k]:
            conOp = ">"
        if "<" in predConNew[k]:
            conOp = "<"
        if "<=" in predConNew[k]:
            conOp = "<="
        if ">=" in predConNew[k]:
            conOp = ">="
        conVal = re.split("=|<|>", predConNew[k])
        conVal = list(filter(None, conVal))
        conCol = conVal[0]
        conColArr = re.split('\.|\s', conCol)
        conColArr = list(filter(None, conColArr))
        try:
            pool_ = db_meta[conColArr[0].upper()][conColArr[1].upper()]
        except:
            sql_rec_logic.append(["Error", real_sql])
            continue
        conVal = re.split('"|\s', conVal[-1])
        conVal = list(filter(None, conVal))
        conVal = ' '.join(conVal)
        predConNew[k] = conCol + conOp + ' "' + conVal + '"'

    pred = predAgg + ' where ' + ' and '.join(predConNew)
    pred = re.split("\s", pred)
    pred = list(filter(None, pred))
    pred = " ".join(pred)

    sql_rec_logic.append([pred, real_sql])
    try:
        myres = model.execute_sql(pred).fetchall()
        myres = list({k[0]: {} for k in myres if not k[0] == None})
        cnt += 1
    except:
        pass
    
generated_sql_file.close()

In [26]:
correct = 0
for itm in sql_rec_logic:
    if itm[0] == itm[1]:
        correct += 1
print(f'Logic Form Accuracy: {round(correct/total_samples, 4)}')

Logic Form Accuracy: 0.595


## Execution Accuracy

In [27]:
generated_sql_file = open('../src/data/generated-sql/predicted-template-sql.json', 'r')

cnt = 0
lblb = 0
sql_rec_exec = []
for line in generated_sql_file:
    line = json.loads(line)
    pred = re.split('<stop>', line['pred_sql'])[0]
    ttt = line['input_sql']

#     print(pred)
    predArr = re.split("where", pred)
    predAgg = re.split("\s", predArr[0])
    predAgg = list(filter(None, predAgg))
    predAgg2 = []
    for k in range(len(predAgg)-1):
        if predAgg[k] in headerDic and predAgg[k+1] in headerDic:
            predAgg2.append(predAgg[k] + ',')
        else:
            predAgg2.append(predAgg[k])
    predAgg2.append(predAgg[-1])
    predAgg = ' '.join(predAgg2)
    
    predCon = re.split("and", predArr[1])
    predConNew = []
    k = 0
    while k < len(predCon):
        if "=" in predCon[k] or "<" in predCon[k] or ">" in predCon[k]:
            predConNew.append(predCon[k])
        else:
            if(len(predConNew) > 0):
                predConNew[-1] += " and " + predCon[k]
            k += 1
        k += 1
    for k in range(len(predConNew)):
        if "=" in predConNew[k]:
            conOp = "="
        if ">" in predConNew[k]:
            conOp = ">"
        if "<" in predConNew[k]:
            conOp = "<"
        if "<=" in predConNew[k]:
            conOp = "<="
        if ">=" in predConNew[k]:
            conOp = ">="
        conVal = re.split("=|<|>", predConNew[k])
        conVal = list(filter(None, conVal))
        conCol = conVal[0]
        conColArr = re.split('\.|\s', conCol)
        conColArr = list(filter(None, conColArr))
        try:
            pool_ = db_meta[conColArr[0].upper()][conColArr[1].upper()]
        except:
            sql_rec_exec.append(["Error", ttt])
            lblb = 1
            break
        conVal = re.split('"|\s', conVal[-1])
        conVal = list(filter(None, conVal))
        conVal = ' '.join(conVal)
        try:
            int(conVal)
            predConNew[k] = conCol + conOp + ' "' + conVal + '"'
        except:
            predConNew[k] = 'lower(' + conCol + ')' + conOp + ' "' + conVal + '"'
    
    if lblb ==1:
        lblb = 0
        continue
    pred = predAgg + ' where ' + ' and '.join(predConNew)
    pred = re.split("\s", pred)
    pred = list(filter(None, pred))
    pred = " ".join(pred)
#     print(pred)
#     print(ttt)
#     print()
    sql_rec_exec.append([pred, ttt])
    try:
        myres = model.execute_sql(pred).fetchall()
        myres = list({k[0]: {} for k in myres if not k[0] == None})
        cnt += 1
    except:
        pass
    
generated_sql_file.close()
print(cnt)

842


In [28]:
fp = open("../src/data/mimicsql_data/mimicsql_template/test.json", 'r')
cnt = 0
for line in fp:
    data = json.loads(line)
    sql_rec_exec[cnt][1] = data['sql']
    cnt += 1
fp.close()

In [29]:
correct = 0
for itm in sql_rec_exec:
    [pred, real_sql] = itm
    try:
        outPred = model.execute_sql(pred).fetchall()
        outTtt = model.execute_sql(real_sql).fetchall()
    except:
        continue
    if outPred == outTtt:
        correct += 1
print(f'Execution Accuracy: {round(correct/total_samples, 4)}')

## Logic Form Accuracy + Recover

In [ ]:
rouge = RougeCalculator(stopwords=False, lang="en")

In [ ]:
def find_best(input_, pool_):
    score_ = []
    for itm in pool_:
        input_ = input_.lower()
        itm = str(itm).lower()
        score_.append(rouge.rouge_n(summary=input_, references=itm, n=1))
    
    if np.sum(score_) == 0:
        score_ = []
        input2_ = ' '.join(list(input_)).lower()
        for itm in pool_:
            itm2 = ' '.join(list(str(itm))).lower()
            score_.append(rouge.rouge_n(summary=input2_, references=itm2, n=1))
    
    return str(pool_[np.argmax(score_)])

In [ ]:
generated_sql_file = open('../src/data/generated-sql/predicted-template-sql.json', 'r')

cnt = 0
lblb = 0
sql_rec_recover = []
for line in generated_sql_file:
    line = json.loads(line)

    pred = re.split('<stop>', line['pred_sql'])[0]
    real_sql = line['input_sql']

    predArr = re.split("where", pred)
    predAgg = re.split("\s", predArr[0])
    predAgg = list(filter(None, predAgg))
    predAgg2 = []
    for k in range(len(predAgg)-1):
        if predAgg[k] in headerDic and predAgg[k+1] in headerDic:
            predAgg2.append(predAgg[k] + ',')
        else:
            predAgg2.append(predAgg[k])
    predAgg2.append(predAgg[-1])
    predAgg = ' '.join(predAgg2)
    
    predCon = re.split("and", predArr[1])
    predConNew = []
    k = 0
    while k < len(predCon):
        if "=" in predCon[k] or "<" in predCon[k] or ">" in predCon[k]:
            predConNew.append(predCon[k])
        else:
            if(len(predConNew) > 0):
                predConNew[-1] += " and " + predCon[k]
            k += 1
        k += 1
    for k in range(len(predConNew)):
        if "=" in predConNew[k]:
            conOp = "="
        if ">" in predConNew[k]:
            conOp = ">"
        if "<" in predConNew[k]:
            conOp = "<"
        if "<=" in predConNew[k]:
            conOp = "<="
        if ">=" in predConNew[k]:
            conOp = ">="
        conVal = re.split("=|<|>", predConNew[k])
        conVal = list(filter(None, conVal))
        conCol = conVal[0]
        conColArr = re.split('\.|\s', conCol)
        conColArr = list(filter(None, conColArr))
        try:
            pool_ = db_meta[conColArr[0].upper()][conColArr[1].upper()]
        except:
            sql_rec_recover.append(["Error", real_sql])
            lblb = 1
            break
        conVal = re.split('"|\s', conVal[-1])
        conVal = list(filter(None, conVal))
        conVal = ' '.join(conVal)
        predConNew[k] = conCol + conOp + ' "' + find_best(conVal, pool_) + '"'
    if lblb == 1:
        lblb = 0
        continue

    pred = predAgg + ' where ' + ' and '.join(predConNew)
    pred = re.split("\s", pred)
    pred = list(filter(None, pred))
    pred = " ".join(pred)
    # print("pred",pred)
    # print("real_sql",real_sql)
    # print("\n")
    sql_rec_recover.append([pred, real_sql])
    try:
        myres = model.execute_sql(pred).fetchall()
        myres = list({k[0]: {} for k in myres if not k[0] == None})
        cnt += 1
    except:
        pass
    
generated_sql_file.close()
print(cnt)

842


In [ ]:
fout = open('../src/data/generated-sql/recovered_template_output.json', 'w')
for itm in sql_rec_recover:
    tmp = {'sql_true': itm[1], 'sql_pred_recovered': itm[0]}
    data = json.dumps(tmp)
    fout.write(data+'\n')
fout.close()

In [ ]:
correct = 0
for itm in sql_rec_recover:
    arr = re.split(',|\s', itm[0].lower())
    arr = list(filter(None, arr))
    arr = ' '.join(arr)
    if arr == itm[1]:
        correct += 1
    
print(f'Logic Form Accuracy + Recover: {correct/total_samples}')

Logic Form Accuracy + Recover: 0.239


## Execution Accuracy + Recover

In [ ]:
fp = open("../src/data/mimicsql_data/mimicsql_template/test.json", 'r')

cnt = 0
for line in fp:
    data = json.loads(line)
    sql_rec_recover[cnt][1] = data['sql']
    cnt += 1
fp.close()

In [ ]:
correct = 0
for itm in sql_rec_recover:
    [pred, real_sql] = itm
    try:
        outPred = model.execute_sql(pred).fetchall()
        outTtt = model.execute_sql(real_sql).fetchall()
    except:
        continue
    if outPred == outTtt:
        correct += 1
print(f'Execution Accuracy + Recover: {round(correct/total_samples, 4)}')

Execution Accuracy + Recover: 0.394
